Parametros

In [35]:
box_size = (100, 100)
page_size = (794, 1123)
cantidad_casos = 500
numeracion_inicial = 5001
white_line_size = 10
generar_errores = False
forzar_decision = True

path_destino = 'gdrive/My Drive/Investigaciones/Cursogramas/casos/'
path_simbolos = 'gdrive/My Drive/Investigaciones/Cursogramas/simbolos/'
path_reglas = 'gdrive/My Drive/Investigaciones/Cursogramas/reglas.json'

Librerias

In [2]:
import os
import os.path
import json
import numpy as np
import xml.etree.cElementTree as ET

from os.path import isfile, join
from PIL import Image, ImageDraw, ImageFont
from random import choice, randint, shuffle

Acceso drive

In [3]:
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


Funciones de documentos

In [20]:
def crear_documento(nombre):
  xml_annotation = generar_documento_xml(nombre)
  page = Image.new('RGB', page_size, (255, 255, 255))

  return page, xml_annotation

def agregar_simbolo(row, col, page, simbolo):
  img_simbolo = Image.open(join(path_simbolos, simbolo))
  posicion = (int(col * box_size[0]) + int(0.5*(box_size[0] - img_simbolo.size[0])), int(row * box_size[1]) + int(0.5*(box_size[1] - img_simbolo.size[1])))
  page.paste(img_simbolo, posicion)

def generar_documento_xml(nombre):
  nombre += ".png"
  xml_annotation = ET.Element('annotation')
  ET.SubElement(xml_annotation, 'folder').text = path_destino
  ET.SubElement(xml_annotation, 'filename').text = nombre
  ET.SubElement(xml_annotation, 'path').text = path_destino + nombre

  xml_source = ET.SubElement(xml_annotation, 'source')
  ET.SubElement(xml_source, 'database').text = 'Unknown'

  xml_size = ET.SubElement(xml_annotation, 'size')
  ET.SubElement(xml_size, 'width').text = str(page_size[0])
  ET.SubElement(xml_size, 'height').text = str(page_size[1])
  ET.SubElement(xml_size, 'depth').text = str(3)

  ET.SubElement(xml_annotation, 'segmented').text = '0'

  return xml_annotation

def agregar_anotacion(row, col, xml_annotation, simbolo):
  width, height = Image.open(join(path_simbolos, simbolo)).size
  simbolo = simbolo.replace(".png", "")

  object = ET.SubElement(xml_annotation, 'object')
  ET.SubElement(object, 'name').text = simbolo
  ET.SubElement(object, 'pose').text = 'Unspecified'
  ET.SubElement(object, 'truncated').text = '0'
  ET.SubElement(object, 'difficult').text = '0'

  xmin = int(col * box_size[0]) + int(0.5*(box_size[0] - width))
  ymin = int(row * box_size[1]) + int(0.5*(box_size[1] - height))
  xmax = int(col * box_size[0]) + width + int(0.5*(box_size[0] - width))
  ymax = int(row * box_size[1]) + height + int(0.5*(box_size[1] - height))
  
  if "horizontal" in simbolo:
    ymin -=  20
    ymax += 20

  if "vertical" in simbolo:
    xmin -= 20
    xmax += 20
  
  xml_bndbox = ET.SubElement(object, 'bndbox')
  ET.SubElement(xml_bndbox, 'xmin').text = str(xmin)
  ET.SubElement(xml_bndbox, 'ymin').text = str(ymin)
  ET.SubElement(xml_bndbox, 'xmax').text = str(xmax)
  ET.SubElement(xml_bndbox, 'ymax').text = str(ymax)

def agregar_anotacion_temporalidad(posicion_y, xml_annotation):
  object = ET.SubElement(xml_annotation, 'object')
  ET.SubElement(object, 'name').text = "temporalidad"
  ET.SubElement(object, 'pose').text = 'Unspecified'
  ET.SubElement(object, 'truncated').text = '0'
  ET.SubElement(object, 'difficult').text = '0'

  xml_bndbox = ET.SubElement(object, 'bndbox')
  ET.SubElement(xml_bndbox, 'xmin').text = str(0)
  ET.SubElement(xml_bndbox, 'ymin').text = str(posicion_y)
  ET.SubElement(xml_bndbox, 'xmax').text = str(page_size[0])
  ET.SubElement(xml_bndbox, 'ymax').text = str(posicion_y + 50)

def marcar_documento_error():
  global nombre, xml_annotation
  nombre += + "_E"
  xml_annotation.find("filename").text = nombre + ".png"
  xml_annotation.find("path").text = path_destino + nombre + ".png"

def guardar_documento(nombre, page, xml_annotation):
  page.save(path_destino + nombre + ".png")
  xml_document = ET.ElementTree(xml_annotation)
  xml_document.write(path_destino + nombre + ".xml" )

Funciones generales

In [5]:
def inicializar_carpeta_destino():
  if os.path.isdir(path_destino):
    files = os.listdir(path_destino)
    for file in files:
      os.remove(path_destino + file)
  else:
    os.makedirs(path_destino)

def cargar_reglas():
  with open(path_reglas) as f:
    reglas = json.load(f)

  return reglas

Funciones de construccion

In [36]:
def elegir_inicio(reglas, externo):
  inicios = [regla for regla in reglas if regla["origen"] == ""]
  inicios = inicios if not externo else [inicio for inicio in inicios if not inicio["exclusivo_interno"]]
  inicios = inicios if posicion[1] == 0 else [inicio for inicio in inicios if not "conector" in inicio["destino"]]

  shuffle(inicios)
  inicio = choice(np.array(inicios))

  return inicio

def obtener_reglas_posibles(origen):
  global reglas, areas_generadas
  area_actual = areas_generadas[-1]

  reglas_posibles = [regla for regla in reglas if regla["origen"] == origen]
  reglas_posibles = reglas_posibles if not area_actual["externo"] else [regla for regla in reglas_posibles if not regla["exclusivo_interno"] ]
  reglas_posibles = reglas_posibles if generar_errores else [regla for regla in reglas_posibles if not regla["error"]]
  shuffle(reglas_posibles)

  return reglas_posibles

def elegir_accion(origen):
  global posicion
  accion = None
  termino = False
  reglas_posibles = obtener_reglas_posibles(origen)
  
  while not termino:
    regla = choice(np.array(reglas_posibles)) if reglas_posibles else None
    regla_aplicable, movimientos = puedo_aplicar_reglar(regla) if regla is not None else (False, [])

    if regla is not None and (not regla_aplicable or (posicion[1] == 0 and regla["destino"] == "")):
      reglas_posibles.remove(regla)
    else:
      accion = {
          "regla": regla,
          "movimientos": movimientos
      }
      termino = True
  
  return accion

def posiciones_usadas(posicion_transicion, posicion_simbolo):
  global areas_generadas

  area_actual = areas_generadas[-1]
  columna_otra_area = False
  posicion_usada = any(simbolo["posicion"] == posicion_transicion or simbolo["posicion"] == posicion_simbolo for simbolo in area_actual["simbolos"])
  for area in areas_generadas:
    if area != area_actual:
      columna_otra_area = any(simbolo["posicion"][0] == posicion_transicion[0] or simbolo["posicion"][0] == posicion_simbolo[0] for simbolo in area["simbolos"])
      if columna_otra_area:
        puede_aplicar = False
        break

  return posicion_usada, columna_otra_area

def puede_aplicar_decision(movimientos):
  global posicion, posicion_limite, areas_generadas

  caminos_decision = []
  posible_posicion = tuple(np.asarray(posicion) + np.asarray(movimientos[0]))
  posible_posicion = tuple(np.asarray(posible_posicion) + np.asarray(movimientos[1]))

  movimiento = (0, 1)
  posicion_transicion = tuple(np.asarray(posible_posicion) + np.asarray(movimiento))
  posicion_simbolo = tuple(np.asarray(posicion_transicion) + np.asarray(movimiento))
  posicion_usada, columna_otra_area = posiciones_usadas(posicion_transicion, posicion_simbolo)

  puede_usar = not (posicion_usada or columna_otra_area) and posicion_simbolo[1] <= posicion_limite[1]
  if not puede_usar:
    movimiento = (choice([-1, 1]), 0)
    posicion_transicion = tuple(np.asarray(posible_posicion) + np.asarray(movimiento))
    posicion_simbolo = tuple(np.asarray(posicion_transicion) + np.asarray(movimiento))
    posicion_usada, columna_otra_area = posiciones_usadas(posicion_transicion, posicion_simbolo)

    puede_usar = not (posicion_usada or columna_otra_area) and posicion_simbolo[0] <= posicion_limite[0] and posicion_simbolo[0] >= 0
    if puede_usar:
      caminos_decision.append(
          {
              "transicion": True,
              "simbolo": "transicion_dummy",
              "posicion": posicion_transicion
          }
      )
      caminos_decision.append(
          {
              "transicion": False,
              "simbolo": "simbolo_dummy",
              "posicion": posicion_simbolo
          }
      )

      movimiento = (-1, 0) if movimiento == (1, 0) else (1, 0)
      posicion_transicion = tuple(np.asarray(posible_posicion) + np.asarray(movimiento))
      posicion_simbolo = tuple(np.asarray(posicion_transicion) + np.asarray(movimiento))
      posicion_usada, columna_otra_area = posiciones_usadas(posicion_transicion, posicion_simbolo)
      puede_usar = not (posicion_usada or columna_otra_area) and posicion_simbolo[0] <= posicion_limite[0] and posicion_simbolo[0] >= 0

      if puede_usar:
        caminos_decision.append(
          {
              "transicion": True,
              "simbolo": "transicion_dummy",
              "posicion": posicion_transicion
          }
        )
        caminos_decision.append(
            {
                "transicion": False,
                "simbolo": "simbolo_dummy",
                "posicion": posicion_simbolo
            }
        )
  else:
    caminos_decision.append(
        {
            "transicion": True,
            "simbolo": "transicion_dummy",
            "posicion": posicion_transicion
        }
    )
    caminos_decision.append(
        {
            "transicion": False,
            "simbolo": "simbolo_dummy",
            "posicion": posicion_simbolo
        }
    )

    movimiento = (choice([-1, 1]), 0)
    posicion_transicion = tuple(np.asarray(posible_posicion) + np.asarray(movimiento))
    posicion_simbolo = tuple(np.asarray(posicion_transicion) + np.asarray(movimiento))
    posicion_usada, columna_otra_area = posiciones_usadas(posicion_transicion, posicion_simbolo)

    puede_usar = not (posicion_usada or columna_otra_area) and posicion_simbolo[0] <= posicion_limite[0] and posicion_simbolo[0] >= 0
    if not puede_usar:
      movimiento = (-1, 0) if movimiento == (1, 0) else (1, 0)
      posicion_transicion = tuple(np.asarray(posible_posicion) + np.asarray(movimiento))
      posicion_simbolo = tuple(np.asarray(posicion_transicion) + np.asarray(movimiento))
      posicion_usada, columna_otra_area = posiciones_usadas(posicion_transicion, posicion_simbolo)
      puede_usar = not (posicion_usada or columna_otra_area) and posicion_simbolo[0] <= posicion_limite[0] and posicion_simbolo[0] >= 0

      if puede_usar:
        caminos_decision.append(
            {
                "transicion": True,
                "simbolo": "transicion_dummy",
                "posicion": posicion_transicion
            }
        )
        caminos_decision.append(
            {
                "transicion": False,
                "simbolo": "simbolo_dummy",
                "posicion": posicion_simbolo
            }
        )

    else:
      caminos_decision.append(
          {
              "transicion": True,
              "simbolo": "transicion_dummy",
              "posicion": posicion_transicion
          }
      )
      caminos_decision.append(
          {
              "transicion": False,
              "simbolo": "simbolo_dummy",
              "posicion": posicion_simbolo
          }
      )

  if puede_usar:
    areas_generadas[-1]["simbolos"].extend(caminos_decision)

  return puede_usar

def puedo_aplicar_reglar(regla):
  global posicion, posicion_limite, areas_generadas

  movimientos = []
  if "actualizacion" in regla["transicion"] or "consulta" in regla["transicion"]:
    movimiento = (1, 0)
    movimientos.append(movimiento)
    movimiento = (1, 0) if "actualizacion" in regla["transicion"] else (0, -1)
    movimientos.append(movimiento)

    posicion_transicion = tuple(np.asarray(posicion) + np.asarray(movimientos[0]))
    posicion_simbolo = tuple(np.asarray(posicion_transicion) + np.asarray(movimientos[1]))

    posicion_usada, columna_otra_area = posiciones_usadas(posicion_transicion, posicion_simbolo)
    puede_aplicar = not (posicion_usada or columna_otra_area) and posicion_simbolo[0] <= posicion_limite[0] and posicion_simbolo[0] >= 0

    return puede_aplicar, movimientos

  if "decision" in regla["origen"]:
    area_actual = areas_generadas[-1]
    simbolos_decision = [simbolo for simbolo in area_actual["simbolos"] if "dummy" in simbolo["simbolo"]]
    movimiento = tuple(np.asarray(simbolos_decision[1]["posicion"]) - np.asarray(simbolos_decision[0]["posicion"]))
    if ("vertical" in regla["transicion"] and movimiento[1] == 1) or ("horizontal" in regla["transicion"] and abs(movimiento[0]) == 1):
        movimientos.append(movimiento)
        movimientos.append(movimiento)

        return True, movimientos

    return False, movimientos

  if "vertical" in regla["transicion"]:
    movimiento = (0, 1)
    movimientos.append(movimiento)
    movimientos.append(movimiento)

    posicion_transicion = tuple(np.asarray(posicion) + np.asarray(movimientos[0]))
    posicion_simbolo = tuple(np.asarray(posicion_transicion) + np.asarray(movimientos[1]))
    posicion_usada, columna_otra_area = posiciones_usadas(posicion_transicion, posicion_simbolo)
    puede_aplicar = (not posicion_usada) and posicion_simbolo[1] <= posicion_limite[1]

    if puede_aplicar and not generar_errores and "proceso" in regla["origen"]:
      area_actual = areas_generadas[-1]
      simbolo_anterior = area_actual["simbolos"][-3]["simbolo"] if len(area_actual["simbolos"]) >= 3 else ""
      puede_aplicar = False if "documento" in simbolo_anterior and "documento" in regla["destino"] else True

    if puede_aplicar and ("archivo" in regla["origen"] or "proceso" in regla["origen"]):
      area_actual = areas_generadas[-1]
      puede_aplicar = False if len(area_actual["simbolos"]) >= 3 and (not "temporalidad" in area_actual["simbolos"][-2]["simbolo"]) else True

    if puede_aplicar and "decision" in regla["destino"]:
      puede_aplicar = puede_aplicar_decision(movimientos)

    return puede_aplicar, movimientos

  if "horizontal" in regla["transicion"]:
    columna = choice([-1, 1])
    movimiento = (columna, 0)

    columnas_cruzar = 1
    if regla["cambio_area"]:
      area_actual = areas_generadas[-1]
      columnas_usadas = [simbolo["posicion"][0] for simbolo in area_actual["simbolos"] if not simbolo["transicion"]]
      columnas_cruzar = int(round(posicion[0] - min(columnas_usadas) if columna == -1 else max(columnas_usadas) - posicion[0], 0)) + 1
      simbolos_usados = len([simbolo for simbolo in area_actual["simbolos"] if not simbolo["transicion"]])

      #es para evitar la transicion de area cuando hay un solo simbolo
      if simbolos_usados <= 1:
        return False, movimientos

    movimientos.append(movimiento)

    puede_aplicar = False
    posicion_prueba = posicion
    for i in range(columnas_cruzar):
      movimientos.append(movimiento)

      posicion_transicion = tuple(np.asarray(posicion_prueba) + np.asarray(movimientos[-2]))
      posicion_simbolo = tuple(np.asarray(posicion_transicion) + np.asarray(movimientos[-1]))
      posicion_usada, columna_otra_area = posiciones_usadas(posicion_transicion, posicion_simbolo)

      puede_aplicar = not (posicion_usada or columna_otra_area) and posicion_simbolo[0] <= posicion_limite[0] and posicion_simbolo[0] >= 0
      posicion_prueba = posicion_simbolo

      if not puede_aplicar:
        break

    if not puede_aplicar:
      movimientos = []
      columna = 1 if columna == -1 else -1
      movimiento = (columna, 0)

      columnas_cruzar = 1
      if regla["cambio_area"]:
        area_actual = areas_generadas[-1]
        columnas_usadas = [simbolo["posicion"][0] for simbolo in area_actual["simbolos"] if not simbolo["transicion"]]
        columnas_cruzar = int(round(posicion[0] - min(columnas_usadas) if columna == -1 else max(columnas_usadas) - posicion[0], 0)) + 1    

      movimientos.append(movimiento)

      posicion_prueba = posicion
      for i in range(columnas_cruzar):
        movimientos.append(movimiento)

        posicion_transicion = tuple(np.asarray(posicion_prueba) + np.asarray(movimientos[-2]))
        posicion_simbolo = tuple(np.asarray(posicion_transicion) + np.asarray(movimientos[-1]))
        posicion_usada, columna_otra_area = posiciones_usadas(posicion_transicion, posicion_simbolo)

        puede_aplicar = not (posicion_usada or columna_otra_area) and posicion_simbolo[0] <= posicion_limite[0] and posicion_simbolo[0] >= 0
        posicion_prueba = posicion_simbolo

        if not puede_aplicar:
          break

    if puede_aplicar and "decision" in regla["destino"]:
      puede_aplicar = puede_aplicar_decision(movimientos)

    return puede_aplicar, movimientos

  return False, []

def agregar_inicio(inicio):
  global posicion, page, xml_annotation
  agregar_simbolo(posicion[1], posicion[0], page, inicio["destino"])
  agregar_anotacion(posicion[1], posicion[0], xml_annotation, inicio["destino"])

def agregar_accion(accion):
  global posicion, page, xml_annotation, areas_generadas, posicion_limite
  regla = accion["regla"]
  movimientos = accion["movimientos"]

  posicion_original = posicion
  for movimiento in movimientos[:-1]:
    posicion = tuple(np.asarray(posicion) + np.asarray(movimiento))
    agregar_simbolo(posicion[1], posicion[0], page, regla["transicion"])
    agregar_anotacion(posicion[1], posicion[0], xml_annotation, regla["transicion"])

    if "decision" in regla["origen"]:
      simbolos = [simbolo for simbolo in areas_generadas[-1]["simbolos"] if simbolo["simbolo"] == "transicion_dummy" and simbolo["posicion"] == posicion]
      simbolo = simbolos[0]
      simbolo["simbolo"] = regla["transicion"]
    else:
      areas_generadas[-1]["simbolos"].append(
          {
              "transicion": True,
              "simbolo": regla["transicion"],
              "posicion": posicion
          }
      )

  desplazamiento = 0
  if "consulta" in regla["transicion"]:
    desplazamiento = 0.5

  posicion = tuple(np.asarray(posicion) + np.asarray(movimientos[-1]))

  destino = regla["destino"]
  if posicion[1] >= posicion_limite[1] and not "archivo" in regla["destino"] and not "proceso" in regla["destino"] and not "destruccion" in regla["destino"] and not "decision" in regla["destino"]:
    destino = [item for item in reglas if item["destino"] == "" and "conector" in item["origen"]][0]["origen"]

  agregar_simbolo(posicion[1], posicion[0] + desplazamiento, page, destino)
  agregar_anotacion(posicion[1], posicion[0] + desplazamiento, xml_annotation, destino)

  if regla["cambio_area"]:
    areas_generadas.append(
        {
            "externo": not regla["exclusivo_interno"] and posicion[0] == 0,
            "simbolos": []
        }
    )

  if "decision" in regla["origen"]:
    simbolos = [simbolo for simbolo in areas_generadas[-1]["simbolos"] if simbolo["simbolo"] == "simbolo_dummy" and simbolo["posicion"] == posicion]
    simbolo = simbolos[0]
    simbolo["simbolo"] = regla["transicion"]
  else:
    areas_generadas[-1]["simbolos"].append(
        {
            "transicion": False,
            "simbolo": destino,
            "posicion": posicion
        }
    )

  if "actualizacion" in regla["transicion"] or "consulta" in regla["transicion"]:
    posicion = posicion_original

def dibujar_lineas_areas(areas):
  global page
  draw = ImageDraw.Draw(page)

  for area in areas:
    primer_columna = min([simbolo["posicion"][0] for simbolo in area["simbolos"] if not simbolo["transicion"]])
    ultima_columna = max([simbolo["posicion"][0] for simbolo in area["simbolos"] if not simbolo["transicion"]])+1

    posicion_linea = int((0.5 + ultima_columna) * box_size[0])
    draw.line([(posicion_linea, 0), (posicion_linea, page.size[1])], fill=(0, 0, 0), width=2)

    posicion_linea = int((primer_columna - 0.5) * box_size[0])
    draw.line([(posicion_linea, 0), (posicion_linea, page.size[1])], fill=(0, 0, 0), width=2)

  for area in [area for area in areas if area["externo"]]:
    ultima_columna = max([simbolo["posicion"][0] for simbolo in area["simbolos"] if not simbolo["transicion"]])+1
    if area["externo"]:
      max_white_lines = int(page.size[1]/white_line_size)
      posicion_linea = int((0.5 + ultima_columna) * box_size[0])

      for i in range(max_white_lines):
        if i % 2 != 0:
          draw.line([(posicion_linea, i*white_line_size), (posicion_linea, i*white_line_size + white_line_size)], fill=(255, 255, 255), width=2)
  
  del draw

def agregar_temporalidad(posicion_maxima):
  global areas_generadas, page, posicion, reglas, xml_annotation

  posicion = tuple(np.asarray(posicion_maxima) + np.asarray((0, 1)))
  posicion_y_linea = int((0.5 + posicion[1]) * box_size[1])
  draw = ImageDraw.Draw(page)
  draw.line([(0, posicion_y_linea), (page.size[0], posicion_y_linea)], fill=(0, 0, 0), width=2)

  max_white_lines = int(page.size[0]/white_line_size)
  for i in range(max_white_lines):
    if i % 2 != 0:
      draw.line([(i*white_line_size, posicion_y_linea), (i*white_line_size + white_line_size, posicion_y_linea)], fill=(255, 255, 255), width=2)

  usar_label = True if not generar_errores else choice([True, False])
  if usar_label:
    font = ImageFont.truetype(font="/usr/share/fonts/truetype/liberation/LiberationMono-Regular.ttf", size=20)
    draw.text((page.size[0]/2, posicion_y_linea-20),"Text",(0,0,0), font=font)
  else:
    marcar_documento_error()

  areas_generadas[-1]["simbolos"].append(
      {
        "transicion": False,
        "simbolo": "temporalidad",
        "posicion": (0, int((0.5 + posicion[1])))
      }
  )

  agregar_anotacion_temporalidad(posicion_y_linea-20, xml_annotation)

  posicion = tuple(np.asarray(posicion) + np.asarray((0, 1)))
  inicio = elegir_inicio(reglas, areas_generadas[-1]["externo"])
  agregar_inicio(inicio)
  origen = inicio["destino"]
  areas_generadas[-1]["simbolos"].append(
    {
        "transicion": False,
        "simbolo": origen,
        "posicion": posicion
    }
  )

  return origen

Cursograma con reglas

In [37]:
inicializar_carpeta_destino()
reglas = cargar_reglas()
posicion_limite = (int(page_size[0]/box_size[0])-1, int(page_size[1]/box_size[1])-1)

#for caso in range(cantidad_casos):
caso = 0
while caso < cantidad_casos:
  nombre = "curso_" + str(numeracion_inicial+caso)
  posicion = (randint(0, posicion_limite[0]), 0)
  page, xml_annotation = crear_documento(nombre)
  inicio = elegir_inicio(reglas, False)
  agregar_inicio(inicio)
  origen = inicio["destino"]
  areas_generadas = [
    {
        "externo": not inicio["exclusivo_interno"] and posicion[0] == 0,
        "simbolos": [
            {
                "transicion": False,
                "simbolo": origen,
                "posicion": posicion
            }
        ]
    }
  ]
  decisiones = []
  termino = False
  uso_decision = False

  while not termino:
    
    accion = elegir_accion(origen)

    if accion["regla"] is not None and accion["regla"]["destino"] != "":
      if accion["regla"]["error"]:
        marcar_documento_error()
      
      agregar_accion(accion)
      origen = accion["regla"]["origen"] if "actualizacion" in accion["regla"]["transicion"] or "consulta" in accion["regla"]["transicion"] else accion["regla"]["destino"]

      if "decision" in origen:
        uso_decision = True

      if posicion[1] >= posicion_limite[1] and not "archivo" in accion["regla"]["destino"] and not "proceso" in accion["regla"]["destino"] and not "destruccion" in accion["regla"]["destino"]:
        origen = [item for item in reglas if item["destino"] == "" and "conector" in item["origen"]][0]["origen"]
      
    else:
        areas_con_decisiones = [area for area in areas_generadas if any([simbolo for simbolo in area["simbolos"] if "dummy" in simbolo["simbolo"]])]
        if len(areas_con_decisiones) > 0:
          area_con_decision = areas_con_decisiones[0]
          simbolos_decision = [simbolo for simbolo in area_con_decision["simbolos"] if "dummy" in simbolo["simbolo"]]
          
          movimiento_realizado = np.asarray(simbolos_decision[1]["posicion"]) - np.asarray(simbolos_decision[0]["posicion"])
          posicion = tuple(np.asarray(simbolos_decision[0]["posicion"]) - movimiento_realizado)

          origen = "decision.png"
          areas_generadas.remove(area_con_decision)
          areas_generadas.append(area_con_decision)
        else:
          y_maximo = -1
          area_con_maximo = None
          posicion_con_maximo = None
          ultimo_simbolo = None
          for area in areas_generadas:
            for simbolo in area["simbolos"]:
              posicion_area = simbolo["posicion"]

              if posicion_area[1] > y_maximo:
                area_con_maximo = area
                y_maximo = posicion_area[1]
                posicion_con_maximo = posicion_area
                ultimo_simbolo = simbolo["simbolo"]

          if y_maximo < posicion_limite[1]-2 and not "conector" in ultimo_simbolo:
            areas_generadas.remove(area_con_maximo)
            areas_generadas.append(area_con_maximo)
            origen = agregar_temporalidad(posicion_area)
          else:
            termino = True

  if not forzar_decision or uso_decision:
    dibujar_lineas_areas(areas_generadas)
    guardar_documento(nombre, page, xml_annotation)
    caso += 1

print("generacion terminada")

generacion terminada
